### My query was i wanted to see what people thought about the teams in the world series, Its the biggest event right now in america so there should be alot of good data

importing the things i need for this assignement

In [147]:
import pandas as pd
import requests 
import json 
import urllib

making the dataframe able to hold 300 rows


In [148]:
pd.set_option('display.max_rows',100)
pd.options.display.max_rows = 300

calling on the twitter api so we can receive the tweets from twitter, also grabbing the token we need from our external txt file

In [149]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
bt = pd.read_csv('twitter_token_9-22.txt', header = 0)

putting the bearer token in 

In [150]:
header = {'Authorization': 'Bearer {}'.format(bt['Bearer_Token'][0])}

making my query and laying down the tweet fields i want to show up in the dataframe

In [151]:
query_text = '(world series OR baseball)(houston OR astros OR texas OR phillies OR philadelphia OR pennsylvania)lang:en -is:retweet  '
query_encoded = urllib.parse.quote(query_text)
tweet_fields = 'lang,public_metrics,author_id,created_at'
user_fields = 'username'

defining what the query URL will be 

In [152]:
query_url = endpoint +'?query={}&tweet.fields={}&user.fields={}'.format(query_encoded, tweet_fields, user_fields)

making response equal to requests.get a twitter api function

In [153]:
response = requests.get(query_url, headers = header)

In [154]:
response.status_code

200

In [155]:
response_dict = json.loads(response.text)

response data frame top 5

In [156]:
response_df.head()

,author_id,id,lang,edit_history_tweet_ids,public_metrics,created_at,text
0,505303397,1585023311909507072,en,[1585023311909507072],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-10-25T21:39:40.000Z,Who has an extra world series ticket in Housto...
1,54747587,1585023295237558272,en,[1585023295237558272],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-10-25T21:39:36.000Z,if the phillies win the world series do you th...
2,142784301,1585023222411456513,en,[1585023222411456513],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-10-25T21:39:19.000Z,Don’t sleep on the Rangers with Bochy at the h...
3,1553545084775407619,1585023171823968256,en,[1585023171823968256],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-10-25T21:39:07.000Z,@BronxBmbrz Huh? MLB has more variety than NFL...
4,815620814506749952,1585023125590528000,en,[1585023125590528000],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-10-25T21:38:56.000Z,Finish it off with a World Series MVP @bryceha...


gathering a "next query" which is just 10 more tweets that arent the same as the last 10

In [157]:
next_query_url = query_url + "&next_token={}".format(response_dict['meta']['next_token'])

setting next response to the twitter api function again to grab the tweets

In [158]:
next_response = requests.get(next_query_url, headers = header)

In [159]:
next_response.status_code

200

making a for loop to keep using the next response term, if n is greater than 0 then the next token will continue to grab new tweets

In [160]:
def twt_recent_search (query, num_pages, header):
    response_list = []
    next_token = ''
    for n in range(0, num_pages):
        if n > 0:
            this_query = query + "&next_token={}".format(next_token)
        else:
            this_query = query
        
        this_response = requests.get(this_query, headers = header)
        print(this_response.status_code)
        this_response_dict = json.loads(this_response.text)
        response_list.append(this_response_dict)
        next_token = this_response_dict['meta']['next_token']
        
    return response_list

seeing if i get good responses on my query loop, which i do i get 30 200s

In [161]:
my_responses = twt_recent_search(query_url,30, header)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


making results equal to the dataframe from earlier

In [162]:
results_1 = pd.DataFrame.from_records(my_responses)

compiles the list of data from the data frames and defines it as results_1

In [163]:
data_list_of_dfs = [pd.DataFrame(x)for x in data_list]

In [164]:
data_df = pd.concat(data_list_of_dfs)

calling on the data_df variable that i defined in the line before

In [165]:
data_df.to_csv('data.csv')

In [166]:
data_df

,created_at,edit_history_tweet_ids,author_id,public_metrics,text,lang,id
0,2022-10-25T21:19:01.000Z,[1585018114097680386],1179960921277509632,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@BettejeanSpata4 OMG. I am so sorry for your l...,en,1585018114097680386
1,2022-10-25T21:18:53.000Z,[1585018082434715649],164306822,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",My son and I will be attending our first World...,en,1585018082434715649
2,2022-10-25T21:18:51.000Z,[1585018074197291008],189687240,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@honkersmcgee69 @Deep_Fried_Dave @CarasikS The...,en,1585018074197291008
3,2022-10-25T21:18:39.000Z,[1585018022225653762],7228352,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",News from H-Town: Vote here! Who do you think ...,en,1585018022225653762
4,2022-10-25T21:18:39.000Z,[1585018022120415232],1087929376832978949,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","@tedcruz @astros Oh Raphael, it has nothing to...",en,1585018022120415232
5,2022-10-25T21:18:26.000Z,[1585017966965686272],1072317966052679680,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@JohnBradley215 @astros Astros don’t do luck -...,en,1585017966965686272
6,2022-10-25T21:18:14.000Z,[1585017918290415618],128143531,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",The Phillies have taken some local World Serie...,en,1585017918290415618
7,2022-10-25T21:17:57.000Z,[1585017847251492864],1578998395032768513,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@MLB @bryceharper3 @Phillies @bryceharper3 is ...,en,1585017847251492864
8,2022-10-25T21:17:39.000Z,[1585017773146910720],548848385,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Houston Astros superfan Mattress Mack can't lo...,en,1585017773146910720
9,2022-10-25T21:17:35.000Z,[1585017755111018496],3226913492,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",It's crazy how 2 rule changes led to the Phill...,en,1585017755111018496


tells me how many tweets i received back
The data i got back was a jumble of everything, there was people attacking each other, score predictions, heckling the players, dads who dont know how to use twitter talking about their team. what i was wanting to find was how both of the fans from the teams in the world series reacted to their team being in. funny enough alot of it are people rooting for the astros just because they beat the yankees in the semi final. although going into this i knew the topic wouldnt be filled with good data i was curious to see which team had more people rooting for them.

In [168]:
len(data_df.index)

300